#### loading the libraries

In [17]:
import os
import sys
import pyvista as pv
import trimesh as tm
import numpy as np
import topogenesis as tg
import pickle as pk
sys.path.append(os.path.realpath('..\..')) # no idea how or why this is not working without adding this to the path TODO: learn about path etc.
from notebooks.resources import RES as res

#### loading the configuration of the test

In [18]:
# load base lattice CSV file
lattice_path = os.path.relpath('../../data/macrovoxels.csv')
macro_lattice = tg.lattice_from_csv(lattice_path)

# load random configuration for testing
config_path = os.path.relpath('../../data/random_lattice.csv')
configuration = tg.lattice_from_csv(config_path)

# load environment
environment_path = os.path.relpath("../../data/movedcontext.obj") 
environment_mesh = tm.load(environment_path)

# load solar vectors
vectors = pk.load(open("../../data/sunvectors.pk", "rb"))

# load vector intensities
intensity = pk.load(open("../../data/ghrval.pk", "rb"))

#### during optimization, arrays like these will be passed to the function:

In [19]:
variable = [0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 0, 0, 1, 1, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0]

#### calling the objective function

In [31]:
# input is the decision variables, a referenca lattice, the visibility vectors, their magnitude (i.e. global horizontal radiation for p.v.), and a mesh of the environment
# output is the total objective score in wH on the roof, and wH per each surface (voxel roofs)
crit, voxcrit = res.crit_1_PV(variable, macro_lattice, vectors, intensity, environment_mesh)

In [23]:
# # angles may not be completely relevant or useful here
# vector_1 = ray_per_ctr
# vector_2 = nrm_per_ray

# # much faster and intuitive with vg library 
# angles = vg.angle(vector_1, vector_2)  # angle of incidence w.r.t. surface normal
# elevat = 90-angles # elevation angle w.r.t horizontal plane

#### generating mesh

In [24]:
meshes, _, _ = res.construct_horizontal_mesh(configuration, configuration.unit)
roofmesh = tm.util.concatenate(meshes)

#### visualisation

In [25]:
p = pv.Plotter(notebook=True)

configuration.fast_vis(p,False,False,opacity=0.1)
# p.add_arrows(ctr_per_ray, -ray_per_ctr, mag=5, show_scalar_bar=False)
# p.add_arrows(ctr_per_ray, nrm_per_ray, mag=5, show_scalar_bar=False)
# p.add_mesh(roof_mesh)
p.add_mesh(environment_mesh)
p.add_mesh(roofmesh, cmap='fire', scalars=np.repeat(voxcrit,2))
p.add_points(vectors*-300)
# p.add_points(horizontal_test_points)

p.show(use_ipyvtk=True)


ViewInteractiveWidget(height=768, layout=Layout(height='auto', width='100%'), width=1024)

[(934.6849359435321, 889.6664645202169, 1045.7249681860692),
 (1.9444256636511739, -43.074045759664074, 112.98445790618828),
 (0.0, 0.0, 1.0)]